In [ ]:
import numpy as np
import xarray as xr
import os
import glob
from pathlib import Path 
import pathlib
from pprint import pprint
import matplotlib.pyplot as plt
import time
import requests
import s3fs
from os.path import dirname, join
import datetime
import myDate
import netCDF4
from scipy.interpolate import griddata

import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [ ]:
alongtrack_file_dir = Path('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output/')
grids_file_dir = Path('/home/jovyan/Data/ECCO_GMSL/gridding_output/')
main_dir = Path('/home/jovyan/ECCO_GMSL/')
output_dir = Path('/home/jovyan/Data/ECCO_GMSL/gridding_3x1_output/')
output_dir2 = Path('/home/jovyan/Data/ECCO_GMSL/GMSL/')
fig_dir = Path('/home/jovyan/Figures/ECCO_GMSL/test_3x1_mapsdiff/')

In [ ]:
#GMSL from synthetic grids
gmsl_grids = xr.open_dataset(str(output_dir2) + '/GMSL_grids_19922017_noerror_100points.nc')
gmsl_grids_3errors_2cm = xr.open_dataset(str(output_dir2) + '/GMSL_grids_19922017_3errors_2cm_100points.nc')

#GMSL from synthetic 3x1 grids
gmsl_grids_3x1_3errors_2cm = xr.open_dataset(str(output_dir2) + '/GMSL_grids_3x1_3errors_2cm.nc')
gmsl_grids_3x1 = xr.open_dataset(str(output_dir2) + '/GMSL_grids_3x1.nc')
gmsl_grids_3x1_old = xr.open_dataset(str(output_dir2) + '/GMSL_grids_3x1_old.nc')

#GMSL from ECCO
ECCO_global_mean_sea_level = xr.open_dataset(str(output_dir2) + '/GMSL_ECCO_dailyfiles_19922017.nc')

In [ ]:
latgrid=np.arange(-90,90,1)
longrid=np.arange(-180,180,3)
[LOgrid,LAgrid] = np.meshgrid(longrid,latgrid)

In [ ]:
fig,ax=plt.subplots(figsize=(10,5))
(gmsl_grids.gmsl_grids*100-gmsl_grids_3x1_old.gmsl_grids_3x1*100).plot(label='grids-grids 3x1 old');
(gmsl_grids.gmsl_grids*100-gmsl_grids_3x1.gmsl_grids_3x1*100).plot(label='grids-grids 3x1');
plt.grid()
plt.ylabel('(cm)',fontsize=18)
plt.xlabel('')
plt.tick_params(axis='both', which='major', labelsize=18)
fig.suptitle('GMSL simulated grids - grids 3x1 (missing data+random noise+orbit error)', fontsize=16)
plt.legend(ncol=1, loc='lower left',fontsize=12)
fig.subplots_adjust(left=0.13,top=0.93,bottom=0.15,right=0.97)

In [ ]:
fig,ax=plt.subplots(figsize=(10,5))
(gmsl_grids.gmsl_grids*100-ECCO_global_mean_sea_level.gmsl_ecco*100).plot(label='grids-ECCO');
(gmsl_grids_3x1_old.gmsl_grids_3x1*100-ECCO_global_mean_sea_level.gmsl_ecco*100).plot(label='grids 3x1-ECCO old');
(gmsl_grids_3x1.gmsl_grids_3x1*100-ECCO_global_mean_sea_level.gmsl_ecco*100).plot(label='grids 3x1-ECCO');
plt.grid()
plt.ylabel('(cm)',fontsize=18)
plt.xlabel('')
plt.tick_params(axis='both', which='major', labelsize=18)
fig.suptitle('GMSL simulated grids or grids 3x1 - ECCO', fontsize=16)
plt.legend(ncol=2, loc='lower right',fontsize=12)
fig.subplots_adjust(left=0.13,top=0.93,bottom=0.15,right=0.97)

### Spatial trends of grids/grids 3x1 - ECCO

In [ ]:
# grids_files = list(ecco_file_dir.glob('*SSHA_gridded_*nc'))
# grids_files=np.sort(grids_files)
# print(grids_files[0:5])
# print('...')
# print(grids_files[-5:])

### Spatial trends of grids-grids 3x1

In [ ]:
grids_files = list(grids_file_dir.glob('*SSHA_gridded_*nc'))
grids_files=np.sort(grids_files)
print(grids_files[0:5])
print('...')
print(grids_files[-5:])

In [ ]:
grids = xr.open_mfdataset(
        paths=grids_files,
        combine='nested',
        concat_dim='time',
        decode_cf=True,
        compat='override',
        coords='minimal',
        chunks={'time': 1}  
    )   
grids

In [ ]:
[LO,LA]=np.meshgrid(grids.longitude,grids.latitude)
[LOgrid,LAgrid]=np.meshgrid(longrid,latgrid)
LOgrid = np.reshape(LOgrid,np.prod(LOgrid.shape))
LAgrid = np.reshape(LAgrid,np.prod(LAgrid.shape))
LO = np.reshape(LO,np.prod(LO.shape))
LA = np.reshape(LA,np.prod(LA.shape))
grids_regrid=[]
for i in range(0,grids.SSHA.shape[0]):
    data = np.reshape(grids.SSHA[i,:,:].values,np.prod(grids.SSHA[i,:,:].shape))
    grids_regrid_tmp = griddata((LO,LA), data, (LOgrid, LAgrid), method='nearest')
    grids_regrid_tmp = np.reshape(grids_regrid_tmp,(1,len(latgrid),len(longrid)))
    try:
        grids_regrid=np.vstack((grids_regrid,grids_regrid_tmp))
    except:
        grids_regrid=grids_regrid_tmp

In [ ]:
grids_regrid = xr.DataArray(grids_regrid, dims=['time','lat','lon'])
grids_regrid = grids_regrid.assign_coords({'lon':longrid,'lat':latgrid})
grids_regrid.name = 'SSHA'
grids_regrid.attrs['units'] = 'm'
grids_regrid['time']=grids.time
grids_regrid

In [ ]:
fig = plt.figure(figsize=(15,5))
cm = plt.get_cmap('gist_rainbow')
ax=fig.gca()

ax = plt.axes(projection=ccrs.Robinson(central_longitude=-67, globe=None))
ax.gridlines()
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.COASTLINE)

p=ax.pcolor(longrid,latgrid,grids_regrid[1,:,:].squeeze(),transform=ccrs.PlateCarree(),vmin=-0.2,vmax=0.2, cmap='jet')
h=plt.colorbar(p)

In [ ]:
fig = plt.figure(figsize=(15,5))
cm = plt.get_cmap('gist_rainbow')
ax=fig.gca()

ax = plt.axes(projection=ccrs.Robinson(central_longitude=-67, globe=None))
ax.gridlines()
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.COASTLINE)

p=ax.pcolor(grids.longitude,grids.latitude,grids.SSHA[1,:,:],transform=ccrs.PlateCarree(),vmin=-0.2,vmax=0.2, cmap='jet')
h=plt.colorbar(p)

In [ ]:
grids_files_3x1 = list(output_dir.glob('*SSHA_gridded_3x1_*nc'))
grids_files_3x1=np.sort(grids_files_3x1)
print(grids_files_3x1[0:5])
print('...')
print(grids_files_3x1[-5:])

In [ ]:
grids_3x1 = xr.open_mfdataset(
        paths=grids_files_3x1,
        combine='nested',
        concat_dim='time',
        decode_cf=True,
        compat='override',
        coords='minimal',
        chunks={'time': 1}  
    )   
grids_3x1

In [ ]:
fig = plt.figure(figsize=(15,5))
cm = plt.get_cmap('gist_rainbow')
ax=fig.gca()

ax = plt.axes(projection=ccrs.Robinson(central_longitude=-67, globe=None))
ax.gridlines()
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.COASTLINE)

p=ax.pcolor(longrid,latgrid,grids_3x1.SSHA[1,:,:].squeeze(),transform=ccrs.PlateCarree(),vmin=-0.2,vmax=0.2, cmap='jet')
h=plt.colorbar(p)

In [ ]:
diff=(grids_regrid-grids_3x1.SSHA)
diff

In [ ]:
diff_aft2008=diff.sel(time=slice('2012-01-01','2017-12-31'))
diff_bef2008=diff.sel(time=slice('1992-01-01','2011-12-31'))

In [ ]:
for i in range(0,len(diff.time)):
    fig = plt.figure(figsize=(15,5))
    cm = plt.get_cmap('gist_rainbow')
    ax=fig.gca()

    ax = plt.axes(projection=ccrs.Robinson(central_longitude=-67, globe=None))
    ax.gridlines()
    ax.add_feature(cfeature.LAND)
    ax.add_feature(cfeature.COASTLINE)
    fig.suptitle(str(diff.time[i].values))

    p=ax.pcolor(longrid,latgrid,diff[i,:,:].squeeze(),transform=ccrs.PlateCarree(),vmin=-0.03,vmax=0.03, cmap='jet')
    h=plt.colorbar(p)
    plt.savefig(str(fig_dir)+'/diff_grids_3x1_'+str(i).zfill(3)+'.png',dpi=70,transparent=False,facecolor='white')

In [ ]:
fig = plt.figure(figsize=(15,5))
cm = plt.get_cmap('gist_rainbow')
ax=fig.gca()

ax = plt.axes(projection=ccrs.Robinson(central_longitude=-67, globe=None))
ax.gridlines()
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.COASTLINE)

p=ax.pcolor(diff.lon,diff.lat,diff.mean(dim='time'),transform=ccrs.PlateCarree(),vmin=-0.002,vmax=0.002, cmap='jet')
h=plt.colorbar(p)

In [ ]:
fig = plt.figure(figsize=(15,5))
cm = plt.get_cmap('gist_rainbow')
ax=fig.gca()

ax = plt.axes(projection=ccrs.Robinson(central_longitude=-67, globe=None))
ax.gridlines()
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.COASTLINE)

p=ax.pcolor(diff.lon,diff.lat,diff_aft2008.mean(dim='time')*1000,transform=ccrs.PlateCarree(),vmin=-10,vmax=10, cmap='jet')
h=plt.colorbar(p)

In [ ]:
fig = plt.figure(figsize=(15,5))
cm = plt.get_cmap('gist_rainbow')
ax=fig.gca()

ax = plt.axes(projection=ccrs.Robinson(central_longitude=-67, globe=None))
ax.gridlines()
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.COASTLINE)

p=ax.pcolor(diff.lon,diff.lat,diff_bef2008.mean(dim='time')*1000,transform=ccrs.PlateCarree(),vmin=-10,vmax=10, cmap='jet')
h=plt.colorbar(p)

In [ ]:
diff_aft2008_mean=diff_aft2008.mean(dim='time')
diff_bef2008_mean=diff_aft2008.mean(dim='time')
diff_bef2008_mean

In [ ]:
fig = plt.figure(figsize=(15,5))
ax=fig.gca()
ax.plot(diff.lat,diff_aft2008_mean.mean(dim='lon')*1000)

In [ ]:
result = diff.polyfit(dim = "time", deg = 1, skipna=True)
trend_diff = result.polyfit_coefficients.sel(degree=1)*int(diff.time[-1]-diff.time[0])

result = (grids_regrid).polyfit(dim = "time", deg = 1, skipna=True)
trend_grids_regrid = result.polyfit_coefficients.sel(degree=1)*int(diff.time[-1]-diff.time[0])

result = (grids.SSHA).polyfit(dim = "time", deg = 1, skipna=True)
trend_grids = result.polyfit_coefficients.sel(degree=1)*int(diff.time[-1]-diff.time[0])

result = (grids_3x1.SSHA).polyfit(dim = "time", deg = 1, skipna=True)
trend_grids_3x1 = result.polyfit_coefficients.sel(degree=1)*int(diff.time[-1]-diff.time[0])

In [ ]:
result = (diff_aft2008).polyfit(dim = "time", deg = 1, skipna=True)
trend_diff_aft2008 = result.polyfit_coefficients.sel(degree=1)*int(diff_aft2008.time[-1]-diff_aft2008.time[0])

result = (grids_regrid.sel(time=slice('2008-01-01','2017-12-31'))).polyfit(dim = "time", deg = 1, skipna=True)
trend_grids_regrid_aft2008 = result.polyfit_coefficients.sel(degree=1)*int(diff_aft2008.time[-1]-diff_aft2008.time[0])

result = (grids.sel(time=slice('2008-01-01','2017-12-31')).SSHA).polyfit(dim = "time", deg = 1, skipna=True)
trend_grids_aft2008 = result.polyfit_coefficients.sel(degree=1)*int(diff_aft2008.time[-1]-diff_aft2008.time[0])

result = (grids_3x1.sel(time=slice('2008-01-01','2017-12-31')).SSHA).polyfit(dim = "time", deg = 1, skipna=True)
trend_grids_3x1_aft2008 = result.polyfit_coefficients.sel(degree=1)*int(diff_aft2008.time[-1]-diff_aft2008.time[0])

In [ ]:
result = (diff_bef2008).polyfit(dim = "time", deg = 1, skipna=True)
trend_diff_bef2008 = result.polyfit_coefficients.sel(degree=1)*int(diff_bef2008.time[-1]-diff_bef2008.time[0])

result = (grids_regrid.sel(time=slice('1992-01-01','2007-12-31'))).polyfit(dim = "time", deg = 1, skipna=True)
trend_grids_regrid_bef2008 = result.polyfit_coefficients.sel(degree=1)*int(diff_bef2008.time[-1]-diff_bef2008.time[0])

result = (grids.sel(time=slice('1992-01-01','2007-12-31')).SSHA).polyfit(dim = "time", deg = 1, skipna=True)
trend_grids_bef2008 = result.polyfit_coefficients.sel(degree=1)*int(diff_bef2008.time[-1]-diff_bef2008.time[0])

result = (grids_3x1.sel(time=slice('1992-01-01','2007-12-31')).SSHA).polyfit(dim = "time", deg = 1, skipna=True)
trend_grids_3x1_bef2008 = result.polyfit_coefficients.sel(degree=1)*int(diff_bef2008.time[-1]-diff_bef2008.time[0])

In [ ]:
fig = plt.figure(figsize=(15,5))
cm = plt.get_cmap('gist_rainbow')
ax=fig.gca()

ax = plt.axes(projection=ccrs.Robinson(central_longitude=-67, globe=None))
ax.gridlines()
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.COASTLINE)

p=ax.pcolor(diff.lon,diff.lat,trend_diff,transform=ccrs.PlateCarree(),vmin=-0.01,vmax=0.01, cmap='jet')
h=plt.colorbar(p)

In [ ]:
trend_diff.mean().values


In [ ]:
fig = plt.figure(figsize=(15,5))
cm = plt.get_cmap('gist_rainbow')
ax=fig.gca()

ax = plt.axes(projection=ccrs.Robinson(central_longitude=-67, globe=None))
ax.gridlines()
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.COASTLINE)

p=ax.pcolor(diff.lon,diff.lat,trend_diff_aft2008,transform=ccrs.PlateCarree(),vmin=-0.01,vmax=0.01, cmap='jet')
h=plt.colorbar(p)

In [ ]:
fig = plt.figure(figsize=(15,5))
cm = plt.get_cmap('gist_rainbow')
ax=fig.gca()

ax = plt.axes(projection=ccrs.Robinson(central_longitude=-67, globe=None))
ax.gridlines()
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.COASTLINE)

p=ax.pcolor(diff.lon,diff.lat,trend_diff_bef2008,transform=ccrs.PlateCarree(),vmin=-0.01,vmax=0.01, cmap='jet')
h=plt.colorbar(p)

In [ ]:
fig = plt.figure(figsize=(15,5))
cm = plt.get_cmap('gist_rainbow')
ax=fig.gca()

ax = plt.axes(projection=ccrs.Robinson(central_longitude=-67, globe=None))
ax.gridlines()
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.COASTLINE)

p=ax.pcolor(diff.lon,diff.lat,trend_grids_3x1,transform=ccrs.PlateCarree(),vmin=-0.2,vmax=0.2, cmap='jet')
h=plt.colorbar(p)

In [ ]:
fig = plt.figure(figsize=(15,5))
cm = plt.get_cmap('gist_rainbow')
ax=fig.gca()

ax = plt.axes(projection=ccrs.Robinson(central_longitude=-67, globe=None))
ax.gridlines()
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.COASTLINE)

p=ax.pcolor(diff.lon,diff.lat,trend_grids_regrid,transform=ccrs.PlateCarree(),vmin=-0.2,vmax=0.2, cmap='jet')
h=plt.colorbar(p)

In [ ]:
fig = plt.figure(figsize=(15,5))
cm = plt.get_cmap('gist_rainbow')
ax=fig.gca()

ax = plt.axes(projection=ccrs.Robinson(central_longitude=-67, globe=None))
ax.gridlines()
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.COASTLINE)

p=ax.pcolor(diff.lon,diff.lat,trend_grids_3x1_aft2008,transform=ccrs.PlateCarree(),vmin=-0.2,vmax=0.2, cmap='jet')
h=plt.colorbar(p)

In [ ]:
fig = plt.figure(figsize=(15,5))
cm = plt.get_cmap('gist_rainbow')
ax=fig.gca()

ax = plt.axes(projection=ccrs.Robinson(central_longitude=-67, globe=None))
ax.gridlines()
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.COASTLINE)

p=ax.pcolor(diff.lon,diff.lat,trend_grids_regrid_aft2008,transform=ccrs.PlateCarree(),vmin=-0.2,vmax=0.2, cmap='jet')
h=plt.colorbar(p)

In [ ]:
fig = plt.figure(figsize=(15,5))
cm = plt.get_cmap('gist_rainbow')
ax=fig.gca()

ax = plt.axes(projection=ccrs.Robinson(central_longitude=-67, globe=None))
ax.gridlines()
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.COASTLINE)

p=ax.pcolor(diff.lon,diff.lat,trend_grids_3x1_bef2008,transform=ccrs.PlateCarree(),vmin=-0.2,vmax=0.2, cmap='jet')
h=plt.colorbar(p)

In [ ]:
fig = plt.figure(figsize=(15,5))
cm = plt.get_cmap('gist_rainbow')
ax=fig.gca()

ax = plt.axes(projection=ccrs.Robinson(central_longitude=-67, globe=None))
ax.gridlines()
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.COASTLINE)

p=ax.pcolor(diff.lon,diff.lat,trend_grids_regrid_bef2008,transform=ccrs.PlateCarree(),vmin=-0.2,vmax=0.2, cmap='jet')
h=plt.colorbar(p)

### Diff maps

### Compare GMSL

In [ ]:
fig,ax=plt.subplots(figsize=(10,5))
(gmsl_grids_3errors_2cm.gmsl_grids*100-gmsl_grids_3x1_3errors_2cm.gmsl_grids_3x1*100).plot(label='grids-grids 3x1');
plt.grid()
plt.ylabel('(cm)',fontsize=18)
plt.xlabel('')
plt.tick_params(axis='both', which='major', labelsize=18)
fig.suptitle('GMSL simulated grids - grids 3x1', fontsize=16)
plt.legend(ncol=1, loc='lower left',fontsize=12)
fig.subplots_adjust(left=0.13,top=0.93,bottom=0.15,right=0.97)

In [ ]:
fig,ax=plt.subplots(figsize=(10,5))
(gmsl_grids.gmsl_grids*100-gmsl_grids_3x1.gmsl_grids_3x1*100).plot(label='grids-grids 3x1');
plt.grid()
plt.ylabel('(cm)',fontsize=18)
plt.xlabel('')
plt.tick_params(axis='both', which='major', labelsize=18)
fig.suptitle('GMSL simulated grids - grids 3x1 (missing data+random noise+orbit error)', fontsize=16)
plt.legend(ncol=1, loc='lower left',fontsize=12)
fig.subplots_adjust(left=0.13,top=0.93,bottom=0.15,right=0.97)

In [ ]:
fig,ax=plt.subplots(figsize=(10,5))
(gmsl_grids.gmsl_grids*100-ECCO_global_mean_sea_level.gmsl_ecco*100).plot(label='grids-ECCO');
(gmsl_grids_3x1.gmsl_grids_3x1*100-ECCO_global_mean_sea_level.gmsl_ecco*100).plot(label='grids 3x1-ECCO');
plt.grid()
plt.ylabel('(cm)',fontsize=18)
plt.xlabel('')
plt.tick_params(axis='both', which='major', labelsize=18)
fig.suptitle('GMSL simulated grids or grids 3x1 - ECCO', fontsize=16)
plt.legend(ncol=2, loc='lower right',fontsize=12)
fig.subplots_adjust(left=0.13,top=0.93,bottom=0.15,right=0.97)

In [ ]:
fig,ax=plt.subplots(figsize=(10,5))
(gmsl_grids_3errors_2cm.gmsl_grids*100-ECCO_global_mean_sea_level.gmsl_ecco*100).plot(label='grids-ECCO');
(gmsl_grids_3x1_3errors_2cm.gmsl_grids_3x1*100-ECCO_global_mean_sea_level.gmsl_ecco*100).plot(label='grids 3x1-ECCO');
plt.grid()
plt.ylabel('(cm)',fontsize=18)
plt.xlabel('')
plt.tick_params(axis='both', which='major', labelsize=18)
fig.suptitle('GMSL simulated grids or grids 3x1 - ECCO', fontsize=16)
plt.legend(ncol=2, loc='lower right',fontsize=12)
fig.subplots_adjust(left=0.13,top=0.93,bottom=0.15,right=0.97)

### Compare alongtrack vs grids vs 3x1 grids

In [ ]:
ECCO_alongtrack_files = list(alongtrack_file_dir.glob('*ECCO_V4r4_alongtrack_SSH_*nc'))
ECCO_alongtrack_files=np.sort(ECCO_alongtrack_files)
print(ECCO_alongtrack_files)

In [ ]:
f=9000
ECCO_alongtrack_files_cycle=ECCO_alongtrack_files[f:f+10]
alongtrack = xr.open_mfdataset(
        paths=ECCO_alongtrack_files_cycle,
        combine='nested',
        concat_dim='i',
        decode_cf=True,
        compat='override',
        coords='minimal',
        chunks={'time': 1}  
    )  
alongtrack=alongtrack.where((np.abs(alongtrack.lat) <= 66))
SSH_at_xy_ano=alongtrack.SSH_at_xy
ind=np.where(np.isfinite(SSH_at_xy_ano.values))
x=SSH_at_xy_ano.lon.values[ind]
y=SSH_at_xy_ano.lat.values[ind]
data=SSH_at_xy_ano.values[ind]

In [ ]:
fig = plt.figure(figsize=(15,5))
cm = plt.get_cmap('gist_rainbow')
ax=fig.gca()

ax = plt.axes(projection=ccrs.Robinson(central_longitude=-67, globe=None))
ax.gridlines()
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.COASTLINE)

p=ax.scatter(x,y,6,data,transform=ccrs.PlateCarree(),vmin=-0.2,vmax=0.2, cmap='jet')
h=plt.colorbar(p)

In [ ]:
ds = xr.open_dataset(str(output_dir) + '/SSHA_gridded_3x1_2017-05-19.nc')
ds

In [ ]:
fig = plt.figure(figsize=(15,5))
cm = plt.get_cmap('gist_rainbow')
ax=fig.gca()

ax = plt.axes(projection=ccrs.Robinson(central_longitude=-67, globe=None))
ax.gridlines()
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.COASTLINE)

p=ax.pcolor(ds.lon,ds.lat,ds.SSHA,transform=ccrs.PlateCarree(),vmin=-0.2,vmax=0.2, cmap='jet')
h=plt.colorbar(p)

In [ ]:
ds = xr.open_dataset(str(Path('/home/jovyan/Data/ECCO_GMSL/gridding_output/')) + '/SSHA_gridded_2017-05-19.nc')
ds

In [ ]:
fig = plt.figure(figsize=(15,5))
cm = plt.get_cmap('gist_rainbow')
ax=fig.gca()

ax = plt.axes(projection=ccrs.Robinson(central_longitude=-67, globe=None))
ax.gridlines()
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.COASTLINE)

p=ax.pcolor(ds.longitude,ds.latitude,ds.SSHA,transform=ccrs.PlateCarree(),vmin=-0.2,vmax=0.2, cmap='jet')
h=plt.colorbar(p)